In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Step 1: Load Historical Data
def load_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data[['Close']]

# Step 2: Prepare the Data
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(len(scaled_data) - look_back - 1):
        X.append(scaled_data[i:(i + look_back), 0])
        y.append(scaled_data[i + look_back, 0])
    
    return np.array(X), np.array(y), scaler

# Step 3: Build the Model
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Specify the input shape using Input layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))  # Another LSTM layer
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the Model with Early Stopping
def train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(X_val, y_val), 
                        callbacks=[early_stopping], 
                        verbose=1)
    return history

# Step 5: Make Predictions
def make_predictions(model, X_test, scaler):
    predictions = model.predict(X_test)
    return scaler.inverse_transform(predictions)

# Step 6: Plotting
def plot_predictions(dates, y_test, predictions):
    fig = go.Figure()

    # Plot actual values
    fig.add_trace(go.Scatter(x=dates, y=y_test, mode='lines', name='Actual Prices'))

    # Plot predicted values
    fig.add_trace(go.Scatter(x=dates, y=predictions, mode='lines', name='Predicted Prices'))

    # Update layout
    fig.update_layout(title='Stock Price Prediction vs Actual Prices',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      legend_title='Legend')

    fig.show()

# Main Execution
if __name__ == "__main__":
    # Load and prepare data with an extended date range
    data = load_data('NVDA', '2000-01-01', '2024-07-20')  # Extended date range
    look_back = 60
    X, y, scaler = prepare_data(data, look_back=look_back)

    # Split data into training and validation sets chronologically
    split_index = int(len(X) * 0.7)  # 70% for training, 30% for validation
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]

    # Reshape data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Build and train the model with early stopping
    model = build_model((X_train.shape[1], 1))
    history = train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32)  # Include validation data

    # Make predictions on the validation set
    predictions = make_predictions(model, X_val, scaler)

    # Calculate and print metrics
    mse = mean_squared_error(scaler.inverse_transform(y_val.reshape(-1, 1)), predictions)
    print(f'Mean Squared Error: {mse}')

    # Create a DataFrame for plotting
    dates = data.index[-len(X_val):]  # Use dates corresponding to validation set
    y_val = scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
    predictions = predictions.flatten()

    # Plot predictions
    plot_predictions(dates, y_val, predictions)


[*********************100%%**********************]  1 of 1 completed

Epoch 1/100


134/134 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - loss: 1.7929e-05 - val_loss: 0.0019
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 2.0382e-07 - val_loss: 0.0020
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 2.3169e-07 - val_loss: 0.0017
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 1.9216e-07 - val_loss: 0.0015
Epoch 5/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 2.7169e-07 - val_loss: 0.0015
Epoch 6/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 1.6866e-07 - val_loss: 0.0014
Epoch 7/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 1.9525e-07 - val_loss: 0.0012
Epoch 8/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 2.0901e-07 - val_loss: 0.0015
Epoch 9/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 1.8684e-07 - val_loss: 0.0011
Epoch 10/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 1.4077e-07 - val_loss: 0.0012
Epoch 11/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 1.2647e-07 - val_loss: 0.00

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Step 1: Load Historical Data
def load_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data[['Close']]

# Step 2: Prepare the Data
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(len(scaled_data) - look_back - 1):
        X.append(scaled_data[i:(i + look_back), 0])
        y.append(scaled_data[i + look_back, 0])
    
    return np.array(X), np.array(y), scaler

# Step 3: Build the Model
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Specify the input shape using Input layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))  # Another LSTM layer
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the Model with Early Stopping
def train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(X_val, y_val), 
                        callbacks=[early_stopping], 
                        verbose=1)
    return history

# Step 5: Make Predictions
def make_predictions(model, X_test, scaler):
    predictions = model.predict(X_test)
    return scaler.inverse_transform(predictions)

# Step 6: Plotting
def plot_predictions(dates, y_test, predictions):
    fig = go.Figure()

    # Plot actual values
    fig.add_trace(go.Scatter(x=dates, y=y_test, mode='lines', name='Actual Prices'))

    # Plot predicted values
    fig.add_trace(go.Scatter(x=dates, y=predictions, mode='lines', name='Predicted Prices'))

    # Update layout
    fig.update_layout(title='Stock Price Prediction vs Actual Prices',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      legend_title='Legend')

    fig.show()

# Main Execution
if __name__ == "__main__":
    # Load and prepare data with an extended date range
    data = load_data('AAPL', '2000-01-01', '2024-07-20')  # Extended date range
    look_back = 60
    X, y, scaler = prepare_data(data, look_back=look_back)

    # Split data into training and validation sets chronologically
    split_index = int(len(X) * 0.7)  # 70% for training, 30% for validation
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]

    # Reshape data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Build and train the model with early stopping
    model = build_model((X_train.shape[1], 1))
    history = train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32)  # Include validation data

    # Make predictions on the validation set
    predictions = make_predictions(model, X_val, scaler)

    # Calculate and print metrics
    mse = mean_squared_error(scaler.inverse_transform(y_val.reshape(-1, 1)), predictions)
    print(f'Mean Squared Error: {mse}')

    # Create a DataFrame for plotting
    dates = data.index[-len(X_val):]  # Use dates corresponding to validation set
    y_val = scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
    predictions = predictions.flatten()

    # Plot predictions
    plot_predictions(dates, y_val, predictions)


2024-07-20 22:35:48.962399: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-20 22:35:49.087604: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-20 22:35:49.229702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 22:35:49.364470: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 22:35:49.415310: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 22:35:49.637607: I tensorflow/core/platform/cpu_feature_gu

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - loss: 8.2267e-04 - val_loss: 6.3248e-04
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 6.5085e-06 - val_loss: 5.8481e-04
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 6.2689e-06 - val_loss: 8.2827e-04
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 6.2049e-06 - val_loss: 0.0016
Epoch 5/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - loss: 5.9126e-06 - val_loss: 0.0013
Epoch 6/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - loss: 5.5856e-06 - val_loss: 0.0011
Epoch 7/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 6.0893e-06 - val_loss: 0.0012
Epoch 8/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 5.6160e-06 - val_loss: 0.0010
Epoch 9/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 5.0086e-06 - val_loss: 0.0015
Epoch 10/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 4.9349e-06 - val_loss: 5.3816e-04
Epoch 11/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss:

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Step 1: Load Historical Data
def load_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data[['Close']]

# Step 2: Prepare the Data
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(len(scaled_data) - look_back - 1):
        X.append(scaled_data[i:(i + look_back), 0])
        y.append(scaled_data[i + look_back, 0])
    
    return np.array(X), np.array(y), scaler

# Step 3: Build the Model
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Specify the input shape using Input layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))  # Another LSTM layer
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the Model with Early Stopping
def train_model(model, X_train, y_train, X_val, y_val, epochs=200, batch_size=64):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(X_val, y_val), 
                        callbacks=[early_stopping], 
                        verbose=1)
    return history

# Step 5: Make Predictions
def make_predictions(model, X_test, scaler):
    predictions = model.predict(X_test)
    return scaler.inverse_transform(predictions)

# Step 6: Plotting
def plot_predictions(dates, y_test, predictions):
    fig = go.Figure()

    # Plot actual values
    fig.add_trace(go.Scatter(x=dates, y=y_test, mode='lines', name='Actual Prices'))

    # Plot predicted values
    fig.add_trace(go.Scatter(x=dates, y=predictions, mode='lines', name='Predicted Prices'))

    # Update layout
    fig.update_layout(title='Stock Price Prediction vs Actual Prices',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      legend_title='Legend')

    fig.show()

# Main Execution
if __name__ == "__main__":
    # Load and prepare data with an extended date range
    data = load_data('AAPL', '2000-01-01', '2024-07-20')  # Extended date range
    look_back = 60
    X, y, scaler = prepare_data(data, look_back=look_back)

    # Split data into training and validation sets chronologically
    split_index = int(len(X) * 0.9)  # 70% for training, 30% for validation
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]

    # Reshape data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Build and train the model with early stopping
    model = build_model((X_train.shape[1], 1))
    history = train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32)  # Include validation data

    # Make predictions on the validation set
    predictions = make_predictions(model, X_val, scaler)

    # Calculate and print metrics
    mse = mean_squared_error(scaler.inverse_transform(y_val.reshape(-1, 1)), predictions)
    print(f'Mean Squared Error: {mse}')

    # Create a DataFrame for plotting
    dates = data.index[-len(X_val):]  # Use dates corresponding to validation set
    y_val = scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
    predictions = predictions.flatten()

    # Plot predictions
    plot_predictions(dates, y_val, predictions)


[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 23s 86ms/step - loss: 0.0040 - val_loss: 0.0011
Epoch 2/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 76ms/step - loss: 7.2908e-05 - val_loss: 8.5952e-04
Epoch 3/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 72ms/step - loss: 7.4415e-05 - val_loss: 0.0012
Epoch 4/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 68ms/step - loss: 6.1266e-05 - val_loss: 7.2913e-04
Epoch 5/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 74ms/step - loss: 6.4869e-05 - val_loss: 7.9209e-04
Epoch 6/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 19s 67ms/step - loss: 7.2263e-05 - val_loss: 5.7365e-04
Epoch 7/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - loss: 5.9501e-05 - val_loss: 5.7129e-04
Epoch 8/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step - loss: 6.7687e-05 - val_loss: 4.9891e-04
Epoch 9/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - loss: 5.4825e-05 - val_loss: 4.5402e-04
Epoch 10/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 80ms/step - loss: 4.3976e-05 - val_loss: 0.0012
Epoch 11/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Step 1: Load Historical Data
def load_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data[['Close']]

# Step 2: Prepare the Data
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(len(scaled_data) - look_back - 1):
        X.append(scaled_data[i:(i + look_back), 0])
        y.append(scaled_data[i + look_back, 0])
    
    return np.array(X), np.array(y), scaler

# Step 3: Build the Model
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Specify the input shape using Input layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))  # Another LSTM layer
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the Model with Early Stopping
def train_model(model, X_train, y_train, X_val, y_val, epochs=200, batch_size=64):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(X_val, y_val), 
                        callbacks=[early_stopping], 
                        verbose=1)
    return history

# Step 5: Make Predictions
def make_predictions(model, X_test, scaler):
    predictions = model.predict(X_test)
    return scaler.inverse_transform(predictions)

# Step 6: Plotting
def plot_predictions(dates, y_test, predictions):
    fig = go.Figure()

    # Plot actual values
    fig.add_trace(go.Scatter(x=dates, y=y_test, mode='lines', name='Actual Prices'))

    # Plot predicted values
    fig.add_trace(go.Scatter(x=dates, y=predictions, mode='lines', name='Predicted Prices'))

    # Update layout
    fig.update_layout(title='Stock Price Prediction vs Actual Prices',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      legend_title='Legend')

    fig.show()

# Main Execution
if __name__ == "__main__":
    # Load and prepare data with an extended date range
    data = load_data('AAPL', '2000-01-01', '2024-07-20')  # Extended date range
    look_back = 60
    X, y, scaler = prepare_data(data, look_back=look_back)

    # Split data into training and validation sets chronologically
    split_index = int(len(X) * 0.9)  # 70% for training, 30% for validation
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]

    # Reshape data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Build and train the model with early stopping
    model = build_model((X_train.shape[1], 1))
    history = train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32)  # Include validation data

    # Make predictions on the validation set
    predictions = make_predictions(model, X_val, scaler)

    # Calculate and print metrics
    mse = mean_squared_error(scaler.inverse_transform(y_val.reshape(-1, 1)), predictions)
    print(f'Mean Squared Error: {mse}')

    # Create a DataFrame for plotting
    dates = data.index[-len(X_val):]  # Use dates corresponding to validation set
    y_val = scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
    predictions = predictions.flatten()

    # Plot predictions
    plot_predictions(dates, y_val, predictions)


[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - loss: 0.0032 - val_loss: 8.2583e-04
Epoch 2/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 19s 66ms/step - loss: 6.4717e-05 - val_loss: 6.9513e-04
Epoch 3/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step - loss: 6.5783e-05 - val_loss: 0.0011
Epoch 4/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 70ms/step - loss: 5.9919e-05 - val_loss: 5.7720e-04
Epoch 5/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step - loss: 5.1471e-05 - val_loss: 5.2393e-04
Epoch 6/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - loss: 5.1014e-05 - val_loss: 6.0012e-04
Epoch 7/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - loss: 4.1813e-05 - val_loss: 4.4872e-04
Epoch 8/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - loss: 6.6914e-05 - val_loss: 4.1466e-04
Epoch 9/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 70ms/step - loss: 4.5091e-05 - val_loss: 4.6096e-04
Epoch 10/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 71ms/step - loss: 4.0151e-05 - val_loss: 3.6652e-04
Epoch 11/100
172/172 ━━━━━━━━━━━━━━━━

In [5]:
# Load unseen data (e.g., most recent years not included in the training/validation data)
unseen_data = load_data('NVDA', '2023-01-01', '2024-07-20')
X_unseen, y_unseen, _ = prepare_data(unseen_data, look_back=look_back)

# Reshape unseen data for LSTM
X_unseen = np.reshape(X_unseen, (X_unseen.shape[0], X_unseen.shape[1], 1))

# Make predictions on the unseen data
predictions_unseen = make_predictions(model, X_unseen, scaler)

# Calculate and print metrics for unseen data
mse_unseen = mean_squared_error(scaler.inverse_transform(y_unseen.reshape(-1, 1)), predictions_unseen)
print(f'Mean Squared Error on unseen data: {mse_unseen}')

# Create a DataFrame for plotting unseen results
dates_unseen = unseen_data.index[-len(X_unseen):]  # Use dates corresponding to unseen data
y_unseen = scaler.inverse_transform(y_unseen.reshape(-1, 1)).flatten()
predictions_unseen = predictions_unseen.flatten()

# Plot unseen predictions
plot_predictions(dates_unseen, y_unseen, predictions_unseen)

[*********************100%%**********************]  1 of 1 completed

 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Mean Squared Error on unseen data: 18.487679729327034
